In [36]:
import pandas as pd
import re

def clean_detail(detail):
   return str(detail).lower().strip()

def get_return_type(detail):
  detail = clean_detail(detail)
  if "touchback" in detail:
    return "touchback"
  if "fair catch" in detail:
    return "fair catch"
  if "returned by" in detail:
    return "ran"
  raise ValueError(f"No valid result method found for detail: '{detail}'")


def get_punt_yards(detail):
  detail = clean_detail(detail)
  pat = r"punts (\d+) yards"
  match = re.search(pat, detail)

  if not match:
    raise ValueError(f"No valid punt yardage found in the provided details: '{detail}'")

  return match.group(1)

def get_punter(detail):
  detail = clean_detail(detail)
  pat = r"(.*)\s*punts"
  match = re.search(pat, detail)

  if not match:
    raise ValueError(f"No valid punter found in the provided details: '{detail}'")

  return match.group(1).title()

def get_returning_player(detail, return_type):
  detail = clean_detail(detail)
  if return_type == "fair catch":
    pat = r"fair catch by\s*(.*)\s*at\s*\w\w-\d*\s*"
    match = re.search(pat, detail)

    if match:
      return match.group(1).title(), None, None

  if return_type == "ran":
    player_pat = r"returned by\s*(.*)\sfor"
    yards_pat = r"returned by\s*.*\s*for\s*(\d*)\s*yards"
    tackle_pat = r"\(tackle by\s*(.*)\)"
    player_match = re.search(player_pat, detail)
    yards_match = re.search(yards_pat, detail)
    tackle_match = re.search(tackle_pat, detail)

    if player_match and yards_match and tackle_match:
      return player_match.group(1).title(), yards_match.group(1), tackle_match.group(1).title()

  if return_type == "touchback":
    return None, None, None

  return ValueError(f"No valid defending special teams player for result_method={return_type}; detail={detail}")

def get_return_location(full_df, detail_ind):
  return full_df.iloc[detail_ind + 1]["Location"]

df = pd.read_csv("20240905-0820_ravens_chiefs.csv")
punt_df = df[df["Detail"].str.lower().str.contains("punt")]

away_team = df.columns[5]
home_team = df.columns[6]
data = []
for i, row in punt_df.iterrows():
  quarter = row["Quarter"]
  time = row["Time"]
  detail = row["Detail"]
  punter = get_punter(detail)
  punt_location = row["Location"]
  return_type = get_return_type(detail)
  punt_yards = get_punt_yards(detail)
  punt_return_location = get_return_location(df, i)
  returning_player, run_yards, tackler = get_returning_player(detail, return_type)


  data.append( {
    "Away Team": away_team,
    "Home Team": home_team,
    "Quarter": quarter,
    "Time": time,
    "Detail": detail,
    "Punter": punter,
    "Punt Location": punt_location,
    "Punt Return Type": return_type,
    "Punt Yards": punt_yards,
    "Punt Return Location": punt_return_location,
    "Returning Player": returning_player,
    "Run Yards": run_yards,
    "Tackler ": tackler,
  } )

data = pd.DataFrame(data)